Kacper Staroń
298842

## PD4 SVM

Oświadczam, że niniejsza praca stanowiąca podstawę do uznania osiągnięcia efektów uczenia się z przedmiotu Wstęp do Uczenia Maszynowego została wykonana przeze mnie samodzielnie.

## Przygotowanie danych i środowiska

In [1]:
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from sklearn.pipeline import Pipeline

In [2]:
data = pd.read_csv('../../Grupa1/apartments.csv')
data_test = pd.read_csv('../../Grupa1/apartments_test.csv')
data = data.append(data_test)

In [3]:
data.head(4)

,m2.price,construction.year,surface,floor,no.rooms,district
0,5897,1953,25,3,1,Srodmiescie
1,1818,1992,143,9,5,Bielany
2,3643,1937,56,1,2,Praga
3,3517,1995,93,7,3,Ochota


Podział na zbiory testowe i treningowe

In [4]:
X, y = data.drop(['district'], axis = 1), data['district']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

## Dane: appartaments

Najpierw wywołamy SVC z domyślnymi parametrami na nieprzeskalowanych danych.

In [6]:
svm = SVC()

In [7]:
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
accuracy_score(y_pred, y_test)

0.2165

Przygotowanie parametrów do sprawdzenia metodą Random Search

In [8]:
C = np.linspace(1, 10, 10)
degree = np.linspace(1, 10, 10)
gamma = ['auto', 'scale'] + np.linspace(0.1, 1, 10).tolist()
param_grid = dict(C = C, degree = degree, gamma = gamma)

In [9]:
param_grid

{'C': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]),
 'degree': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]),
 'gamma': ['auto',
  'scale',
  0.1,
  0.2,
  0.30000000000000004,
  0.4,
  0.5,
  0.6,
  0.7000000000000001,
  0.8,
  0.9,
  1.0]}

### Nieprzeskalowane dane

In [10]:
random = RandomizedSearchCV(estimator=svm, param_distributions=param_grid, cv = 3, n_jobs=-1)

random_result = random.fit(X_train, y_train)

In [11]:
random_result.best_score_

0.2511254454033456

In [12]:
random_result.best_params_

{'gamma': 'scale', 'degree': 5.0, 'C': 9.0}

Widzimy, że po dostrojeniu modelu wynik uległ niewielkiej, acz wyraźnej poprawie. 

Teraz przeskalujemy dane dla poprawy osiągów algorytmu.

In [13]:
scaler = StandardScaler()

scaler.fit(X)
X_scaled = scaler.transform(X)

X_train_sc, X_test_sc, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)


In [14]:
svm.fit(X_train_sc, y_train)
y_pred = svm.predict(X_test_sc)
accuracy_score(y_pred, y_test)

0.311

Widzimy, że wynik modelu znacząco się zwiększył.

### Przeskalowane dane

In [15]:
random = RandomizedSearchCV(estimator=svm, param_distributions=param_grid, cv = 3, n_jobs=-1)

random_result = random.fit(X_train_sc, y_train)

In [16]:
random_result.best_score_

0.3040001657707265

In [17]:
random_result.best_params_

{'gamma': 0.7000000000000001, 'degree': 10.0, 'C': 3.0}

Po dostrojeniu uzyskaliśmy minimalny spadek skuteczności.

### Sprawdzenie innych jąder

In [23]:
svm = SVC(kernel = 'linear')
svm.fit(X_train_sc, y_train)
y_pred = svm.predict(X_test_sc)
accuracy_score(y_pred, y_test)

0.3065

In [20]:
svm = SVC(kernel = 'poly')
svm.fit(X_train_sc, y_train)
y_pred = svm.predict(X_test_sc)
accuracy_score(y_pred, y_test)

0.3155

In [22]:
svm = SVC(kernel = 'sigmoid')
svm.fit(X_train_sc, y_train)
y_pred = svm.predict(X_test_sc)
accuracy_score(y_pred, y_test)

0.2115

Pobieżny przegląd wyników modeli na innych jądrach z domyślnymi parametrami nie sugeruje żadnego istotnie lepszego rozwiązania niż 'rbf'.

## Dane: EEG

Zbiór danych pochodzi ze strony OpenML i dotyczy pomiarów EEG. Zawiera 14 zmiennych numerycznych (V1 - V14) opisujących parametry medyczne zarejestrowane podczas pomiaru i jedną zmienną kategoryczną "Class" (tutaj zmienna celu) oznaczającą, czy pacjent podczas danego pomiaru miał oczy zamknięte (1) czy otwarte (0).


In [24]:
data = pd.read_csv('https://www.openml.org/data/get_csv/1587924/phplE7q6h')

In [25]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,Class
0,4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.90,4393.85,1
1,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.10,1
2,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,1
3,4328.72,4011.79,4296.41,4155.90,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,1
4,4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.90,4627.69,4210.77,4244.10,4212.82,4288.21,4632.82,4398.46,1


In [26]:
X, y = data.drop(['Class'], axis = 1), data['Class']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [28]:
svm = SVC()

Wywołanie bez strojenia na danych nieprzeskalowanych:

In [29]:
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
accuracy_score(y_pred, y_test)

0.5510680907877169

### Nieprzeskalowane dane

In [31]:
random = RandomizedSearchCV(estimator=svm, param_distributions=param_grid, cv = 3, n_jobs=-1)

random_result = random.fit(X_train, y_train)

In [32]:
random_result.best_score_

0.5552399939082591

In [33]:
random_result.best_params_

{'gamma': 'scale', 'degree': 9.0, 'C': 6.0}

In [34]:
scaler = StandardScaler()

scaler.fit(X)
X_scaled = scaler.transform(X)

X_train_sc, X_test_sc, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)


Wywołanie bez strojenia na danych przeskalowanych:

In [35]:
svm.fit(X_train_sc, y_train)
y_pred = svm.predict(X_test_sc)
accuracy_score(y_pred, y_test)

0.6348464619492656

### Przeskalowane dane

In [36]:
random = RandomizedSearchCV(estimator=svm, param_distributions=param_grid, cv = 3, n_jobs=-1)

random_result = random.fit(X_train_sc, y_train)

In [37]:
random_result.best_score_

0.8151702313587194

In [38]:
random_result.best_params_

{'gamma': 0.9, 'degree': 10.0, 'C': 9.0}

Wyniki na nieprzeskalowanych danych są praktycznie niezależne od strojenia.
Dla danych przeskalowanych obserwujemy istotny wzrost efektywności modelu, a po strojeniu hiperparametrów zadaną metodą otrzymujemy świetny wynik.

### Sprawdzenie innych jąder

In [39]:
svm = SVC(kernel = 'linear')
svm.fit(X_train_sc, y_train)
y_pred = svm.predict(X_test_sc)
accuracy_score(y_pred, y_test)

0.6148197596795728

In [40]:
svm = SVC(kernel = 'poly')
svm.fit(X_train_sc, y_train)
y_pred = svm.predict(X_test_sc)
accuracy_score(y_pred, y_test)

0.5781041388518025

In [41]:
svm = SVC(kernel = 'sigmoid')
svm.fit(X_train_sc, y_train)
y_pred = svm.predict(X_test_sc)
accuracy_score(y_pred, y_test)

0.5670894526034713

Podobnie jak dla pierwszego zbioru danych, pobieżny przegląd wyników modeli na innych jądrach z domyślnymi parametrami nie sugeruje żadnego istotnie lepszego rozwiązania niż 'rbf'.